In [27]:
#In Parallel Network architecture, multiple layers/networks can run parallely; and can be combined together using Concatenation Layer
# see john crown
import keras
from keras.datasets import imdb
from keras.models import Model
from keras.layers import Input, concatenate, Dense, Dropout, Embedding, SpatialDropout1D, Conv1D, GlobalMaxPooling1D
from keras.layers.wrappers import Bidirectional
from keras.preprocessing.sequence import pad_sequences

In [30]:
#Hyperparameters

epochs = 4
batch_size=128

n_dim=64
n_unique_words = 5000
max_review_len = 400 # since it bidir; the capacity is more
pad_type = trunc_type = 'pre'
drop_embed=0.2
 
n_conv1=256  #layers
n_conv2=256
n_conv3=256

k_conv1=3  #focus area / kernel
k_conv2=2
k_conv3=4

n_dense=256 
dropout=0.2

In [31]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words)

In [32]:
x_train = pad_sequences(x_train, maxlen=max_review_len, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_len, padding=pad_type, truncating=trunc_type, value=0)

In [33]:
input_layer = Input(shape=(max_review_len,), dtype='int16', name='input')
embedding_layer=Embedding(n_unique_words, n_dim, input_length=max_review_len, name='embedding')(input_layer)
drop_embed_layer=SpatialDropout1D(drop_embed, name='drop_embed')(embedding_layer)

conv_1=Conv1D(n_conv1,k_conv1, activation='relu', name='conv1')(drop_embed_layer)
maxp_1=GlobalMaxPooling1D(name='maxp_1')(conv_1)

conv_2=Conv1D(n_conv2,k_conv2, activation='relu', name='conv2')(drop_embed_layer)
maxp_2=GlobalMaxPooling1D(name='maxp_2')(conv_2)

conv_3=Conv1D(n_conv3,k_conv3, activation='relu', name='conv3')(drop_embed_layer)
maxp_3=GlobalMaxPooling1D(name='maxp_3')(conv_3)

concat = concatenate([maxp_1,maxp_2,maxp_3])

dense_layer = Dense(n_dense, activation='relu', name='dense')(concat)
drop_dense_layer = Dropout(dropout, name='drop_dense')(dense_layer)

dense_layer2 = Dense(64, activation='relu', name='dense_2')(drop_dense_layer)
drop_dense_layer2 = Dropout(dropout, name='drop_dense_2')(dense_layer2)

predictions = Dense(1, activation='sigmoid', name='output')(drop_dense_layer2)

model= Model(input_layer, predictions)

In [35]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 400)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 400, 64)      320000      input[0][0]                      
__________________________________________________________________________________________________
drop_embed (SpatialDropout1D)   (None, 400, 64)      0           embedding[0][0]                  
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 398, 256)     49408       drop_embed[0][0]                 
__________________________________________________________________________________________________
conv2 (Con

In [36]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 201s 8ms/step - loss: 0.4975 - acc: 0.7286 - val_loss: 0.2947 - val_acc: 0.8773
Epoch 2/4
 5504/25000 [=====>........................] - ETA: 2:01 - loss: 0.2450 - acc: 0.9064

KeyboardInterrupt: 